In [9]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['유사 문장 생성']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    sentence_dict = {}
    
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            new_sentence = {}
            sentence = line['sentence']
            sentence_gen = line['label-scenes']
            score = sum([line['final.grammar'],line['final.reality'],line['final.fluency'],line['final.diversity']])
            new_sentence['score'] = {
                'grammar': line['final.grammar'],
                'reality': line['final.reality'],
                'fluency': line['final.fluency'],
                'diversity': line['final.diversity'],
                'total': score
            }
            new_sentence['text'] = sentence_gen
            if sentence in sentence_dict:
                sentence_dict[sentence].append((score, new_sentence))
            else:
                sentence_dict[sentence] = [(score, new_sentence)]
            #### data preprocess end 

    for sentence, value in tqdm(sentence_dict.items(), desc=split):
        value = sorted(value, reverse=True, key=lambda x: x[0])
        value = [v[1] for v in value if v[1]['text'] != sentence]
        #### 유사 문장 생성
        data = {'input': None, 'output': {}}
        ## preprocess data from line
        data['input'] = sentence
        data['output']['generated_sentences'] = value
        ## preprocess data from line end
        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### 유사 문장 생성 end


            
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 21521/21521 [00:00<00:00, 43079.57it/s]
